In [1]:
""" json.dumps(): 将 Python 对象编码成 JSON 字符串。 json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
    """
import os
import json
def json_standard(input_object):
    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def write_jsonl_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path, "w", encoding="utf-8") as file:
            for item in obj:
                json.dump(item, file, ensure_ascii=False)  # ensure_ascii=False以支持中文字符
                file.write("\n")  # 写入换行符分隔每个JSON对象
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def read_jsonl_file(file_path:str)->list:
    result=[]
    with open(file_path, "r", encoding="utf-8") as file:
        for item in file.readlines():
            item=json.loads(item.strip())  # ensure_ascii=False
            result.append(item)
    return result
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist

In [4]:
eval_arg="base"

In [6]:
from datasets import load_dataset
if eval_arg=="base":
    test_dataset= load_dataset("json",data_files="/home/ckqsudo/code2024/Mistral-7b-finetune/train_and_test_data/test_dataset_pure_dialog.jsonl")
else:
    raise ValueError

In [7]:
test_dataset['train']

Dataset({
    features: ['messages'],
    num_rows: 81
})

In [8]:
def generate_test_format_data(cur_dataset):
    """Explore_data"""
    explore_test_dataset=[]
    for idx,msg_item in enumerate(test_dataset['train']):
        msg_list=msg_item['messages'].copy()
        while True:
            while len(msg_list)!=0 and 'user' in msg_list[-1]['role']:
                msg_list=msg_list[:-1]
            while len(msg_item)!=0 and 'assistant' in msg_list[-1]['role']:
                ground_truth=msg_list[-1]
                dialog_history=msg_list[:-1]
                template={"history":dialog_history,"ground_truth":ground_truth}
                explore_test_dataset.append(template)
                msg_list=msg_list[:-1]
        # 如
            if len(set([item['role'] for item in msg_list]))<2:
                break
    return explore_test_dataset
format_datset=generate_test_format_data(test_dataset)

In [9]:
len(format_datset)

418

In [10]:
format_datset[0].keys()

dict_keys(['history', 'ground_truth'])

In [11]:
print(format_datset[0]['ground_truth'])

{'content': 'Therapist: I see … so … the day after the miracle he would notice that you were not standing there hollering at him … hmm … what do you think he’d see you do instead?', 'role': 'assistant'}


In [14]:
from openai import OpenAI
import os
import time
client = OpenAI()
client.base_url='https://gptnb.keqichen.top/v1'
result=[]

for idx,item in enumerate(format_datset[:]):
  gpt_msg_list=item['history']
  gpt_msg_list[0]['role']="system"
  gpt_3_5_model="gpt-3.5-turbo"
  completion = client.chat.completions.create(
    model=gpt_3_5_model,
    messages=gpt_msg_list
  )
  print("GPT",completion.choices[0].message,"Truth:",item['ground_truth']['content'])
  response_str=completion.choices[0].message.content
  ground_turth=item['ground_truth']['content']
  result.append({
    'hyps':response_str[response_str.find("Therapist:"):],
    'ref': ground_turth[ ground_turth.find("Therapist:"):],
    'annotation':response_str[:response_str.find("Therapist:")],
    'ground_turth': ground_turth,
    "history":item['history']})
  write_jsonl_file("/home/ckqsudo/code2024/Mistral-7b-finetune/metrics/nlg-metrics/Input_for_eval/gpt_3_5_base_pure_dialog.jsonl",result,True)
  time.sleep(0.05)

GPT ChatCompletionMessage(content='Therapist: What do you think Pierre would be wondering about then, when he sees this?', role='assistant', function_call=None, tool_calls=None) Truth: Therapist: I see … so … the day after the miracle he would notice that you were not standing there hollering at him … hmm … what do you think he’d see you do instead?
写入/home/ckqsudo/code2024/Mistral-7b-finetune/metrics/nlg-metrics/Input_for_eval/gpt_3_5_base_pure_dialog.jsonl
GPT ChatCompletionMessage(content='Therapist: How would he be different toward you, Karen?', role='assistant', function_call=None, tool_calls=None) Truth: Therapist: What would Pierre notice?
写入/home/ckqsudo/code2024/Mistral-7b-finetune/metrics/nlg-metrics/Input_for_eval/gpt_3_5_base_pure_dialog.jsonl
GPT ChatCompletionMessage(content='Therapist: How would you ideally like to start your day with your children?', role='assistant', function_call=None, tool_calls=None) Truth: Therapist: What would be the first thing that any of the ki

In [ ]:
write_file("/home/ckqsudo/code2024/Mistral-7b-finetune/metrics/nlg-metrics/Input_for_eval/gpt_3_5.json",result,True)

In [1]:
result

NameError: name 'result' is not defined